In [1]:
from huggingface_hub import notebook_login, interpreter_login
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token can be pasted using 'Right-Click'.


Enter your token (input will not be visible):  ········
Add token as git credential? (Y/n)  n


Token is valid (permission: read).
Your token has been saved to C:\Users\katko\.cache\huggingface\token
Login successful


In [2]:
from transformers.utils import send_example_telemetry

send_example_telemetry("token_classification_notebook", framework="pytorch")

In [3]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

## Preprocessing

In [4]:
from preprocessing import extract_data_from_conll_extended
extract_data_from_conll_extended("data/en_ewt-up-test.conllu", "test.json")

In [5]:
from datasets import load_dataset, load_metric
datasets = load_dataset('json', data_files={'train': 'train.json', 'test': 'test.json'})
datasets

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['arguments', 'predicate', 'word', 'predicate_position'],
        num_rows: 40482
    })
    test: Dataset({
        features: ['arguments', 'predicate', 'word', 'predicate_position'],
        num_rows: 4799
    })
})

In [6]:
datasets["train"][0]

{'arguments': ['_',
  '_',
  '_',
  '_',
  '_',
  'ARG0',
  'V',
  'ARG1',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  'ARGM-LOC',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_'],
 'predicate': 'kill.01',
 'word': ['Al',
  '-',
  'Zaman',
  ':',
  'American',
  'forces',
  'killed',
  'Shaikh',
  'Abdullah',
  'al',
  '-',
  'Ani',
  ',',
  'the',
  'preacher',
  'at',
  'the',
  'mosque',
  'in',
  'the',
  'town',
  'of',
  'Qaim',
  ',',
  'near',
  'the',
  'Syrian',
  'border',
  '.'],
 'predicate_position': 7}

In [7]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(datasets["train"])

,arguments,predicate,word,predicate_position
0,"[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, V, _, _, ARGM-ADJ, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _]",solve.01,"[The, U.S., State, Department, and, the, misnamed, think, tanks, that, follow, its, lead, have, held, numerous, conferences, on, Kashmir, and, ,, in, most, of, them, ,, the, solution, that, has, emerged, is, a, valley, prized, loose, from, Indian, control, and, under, its, own, version, of, Ibrahim, Rugova, .]",28
1,"[ARGM-ADJ, V, _]",serve.02,"[Great, service, .]",2
2,"[_, _, _, _, _, _, _, _, _, _, _, ARG0, _, _, _, V, _, _, _, ARG1, _, _, _]",know.01,"[I, 'm, getting, a, 10, gallon, for, my, betta, tomorrow, and, I, just, want, to, know, how, to, properly, set, it, up, .]",16
3,"[_, _, _, _, _, _, V, _, _, ARG1, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _]",notice.03,"[They, are, deemed, to, be, on, notice, that, they, had, alternate, point, rights, (, it, s, in, the, tariff, ), and, if, they, wanted, to, submit, a, bid, that, had, a, different, rate, for, primaries, and, alternates, ,, they, could, have, done, so, .]",7
4,"[_, _, ARG1, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, V, _, _, _, ARG2, _]",fall.04,"[A, third, report, ,, "", The, Impacts, of, Climate, Change, :, An, Appraisal, for, the, Future, ,, "", completed, by, the, Britain, -, based, International, Policy, Network, and, released, almost, simultaneously, with, the, other, two, ,, falls, somewhat, into, that, category, .]",37
5,"[ARG0, V, ARG1, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _]",make.05,"[I, MAKE, MONEY, NOT, DEAL, WITH, BROKERS, "", Wow, ,, can, you, believe, in, today, s, tough, times, this, dealership, would, be, looking, for, any, way, to, move, vehicles, .]",2
6,"[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ARG2, ARGM-MOD, _, ARG1, _, V, _, _]",send.01,"[Is, there, an, article, to, be, included, in, an, Enron, publication, and, in, addition, is, a, letter, to, be, sent, under, John, 's, name, (, if, so, ,, to, whom, will, the, letter, be, sent, ), ?]",34
7,"[V, _, ARGM-EXT, _]",thank.01,"[Thanks, a, lot, .]",1
8,"[_, _, _, _, _, _, _, _, _, _, _, _, _, _, ARG0, _, V, _, _, _, _, _, _, _]",open_up.03,"[Now, that, Sibley, is, not, a, candidate, for, Lt., Governor, ,, there, is, more, freedom, to, open, up, and, ask, some, difficult, questions, .]",17
9,"[_, _, _, _, _, _, V, _, ARG1, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _]",way.01,"[I, was, not, happy, with, the, way, they, looked, ,, very, wavy, ,, uneven, edges, ,, and, with, the, exception, of, 1, ,, there, is, a, dip, in, the, center, of, each, nail, .]",7


In [9]:
datasets["train"][0]

{'arguments': ['_',
  '_',
  '_',
  '_',
  '_',
  'ARG0',
  'V',
  'ARG1',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  'ARGM-LOC',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_'],
 'predicate': 'kill.01',
 'word': ['Al',
  '-',
  'Zaman',
  ':',
  'American',
  'forces',
  'killed',
  'Shaikh',
  'Abdullah',
  'al',
  '-',
  'Ani',
  ',',
  'the',
  'preacher',
  'at',
  'the',
  'mosque',
  'in',
  'the',
  'town',
  'of',
  'Qaim',
  ',',
  'near',
  'the',
  'Syrian',
  'border',
  '.'],
 'predicate_position': 7}

### Tokenization

In [10]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [11]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [12]:
def tokenize(instance):
    def align(tokenized_input, labels):
        word_ids = tokenized_input.word_ids()
        aligned_labels = ['_'] * len(word_ids) 
        label_index = 0 
        for i, word_id in enumerate(word_ids):
            try:
                if word_id is None:
                    aligned_labels[i] = '[PAD]'
                    continue 
                original_label = labels[word_id]
                if original_label == '_':
                    continue 
                if i == 0 or word_id != word_ids[i-1]:
                    prefix = ''
                else:
                    prefix = ''
                aligned_labels[i] = f'{prefix}{original_label}'
            except:
                pass
    
        return aligned_labels
    sentence = instance["word"]
    labels = instance["arguments"]
    tokenized_input = tokenizer(sentence, is_split_into_words=True)
    predicate = instance['predicate']
    predicate = tokenizer(predicate)
    tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
    aligned_labels = align(tokenized_input, labels)
    tokenized_input['input_ids'].extend(predicate['input_ids'][1:]), tokenized_input['attention_mask'].extend(predicate['attention_mask'][1:]), aligned_labels.extend(['[PAD]' for i in range(len(predicate['input_ids'][1:]))])
    tokenized_input['labels'] = aligned_labels
    
    return tokenized_input


In [13]:
print(len(datasets['train']))
training_set = [tokenize(instance) for instance in datasets['train']]
test_set = [tokenize(instance) for instance in datasets['test']]


40482


In [14]:
sample = training_set[0]
print(sample)

{'input_ids': [101, 2632, 1011, 23564, 2386, 1024, 2137, 2749, 2730, 21146, 28209, 14093, 2632, 1011, 2019, 2072, 1010, 1996, 14512, 2012, 1996, 8806, 1999, 1996, 2237, 1997, 1053, 4886, 2213, 1010, 2379, 1996, 9042, 3675, 1012, 102, 3102, 1012, 5890, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': ['[PAD]', '_', '_', '_', '_', '_', '_', 'ARG0', 'V', 'ARG1', 'ARG1', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', 'ARGM-LOC', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '_', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']}


In [15]:
from datasets import Dataset, Features, Sequence, Value

features = Features({
    "input_ids": Sequence(feature=Value(dtype='int64')),
    "attention_mask": Sequence(feature=Value(dtype='int64')),
    "labels": Sequence(feature=Value(dtype='int64')),
})

In [16]:
unseen = ['R-ARGM-ADJ', '_', 'ARGM-ADJ']
label_list = list(set(label for instance in training_set for label in instance['labels'] + unseen))
label_dict = {label:int(i) for i, label in enumerate(list(label_list))}
label_dict['[PAD]'] = -100
for instance in training_set:
    instance['labels'] = [int(label_dict[label]) for label in instance['labels']]
for instance in test_set:
    instance['labels'] = [int(label_dict[label]) for label in instance['labels']]

In [17]:
dataset_train = Dataset.from_dict({"input_ids": [item['input_ids'] for item in training_set],
                             "attention_mask": [item['attention_mask'] for item in training_set],
                             "labels": [item['labels'] for item in training_set]}, features=features)

dataset_test = Dataset.from_dict({"input_ids": [item['input_ids'] for item in test_set],
                             "attention_mask": [item['attention_mask'] for item in test_set],
                             "labels": [item['labels'] for item in test_set]}, features=features)

In [18]:
#print(tokenized_input.word_ids())
print(dataset_train[0])

{'input_ids': [101, 2632, 1011, 23564, 2386, 1024, 2137, 2749, 2730, 21146, 28209, 14093, 2632, 1011, 2019, 2072, 1010, 1996, 14512, 2012, 1996, 8806, 1999, 1996, 2237, 1997, 1053, 4886, 2213, 1010, 2379, 1996, 9042, 3675, 1012, 102, 3102, 1012, 5890, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 32, 32, 32, 32, 32, 32, 24, 34, 2, 2, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 7, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, -100, -100, -100, -100, -100]}


In [19]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
dataset_train[0]

{'input_ids': [101,
  2632,
  1011,
  23564,
  2386,
  1024,
  2137,
  2749,
  2730,
  21146,
  28209,
  14093,
  2632,
  1011,
  2019,
  2072,
  1010,
  1996,
  14512,
  2012,
  1996,
  8806,
  1999,
  1996,
  2237,
  1997,
  1053,
  4886,
  2213,
  1010,
  2379,
  1996,
  9042,
  3675,
  1012,
  102,
  3102,
  1012,
  5890,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [-100,
  32,
  32,
  32,
  32,
  32,
  32,
  24,
  34,
  2,
  2,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  7,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  32,
  -100,
  -100,
  -100,
  -100,
  -100]}

## Base model

In [21]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

In [22]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [23]:
metric = load_metric("seqeval")


C:\Users\katko\AppData\Local\Temp\ipykernel_25504\2951164050.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
C:\Users\katko\miniconda3\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [24]:
labels = list(label_list)
metric.compute(predictions=[labels], references=[labels])

C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: C-ARG2 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: R-ARGM-GOL seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ARG1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: R-ARG2 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: C-ARGM-LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\katko\miniconda3\Lib\site-packages\

{'ADJ': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ADV': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARG0': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARG1': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARG1-DSP': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARG2': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARG3': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARG4': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARGM-ADJ': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARGM-ADV': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARGM-CAU': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARGM-COM': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARGM-CXN': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ARGM-DIR': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ARGM-EXT': {'precisi

In [25]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [26]:
stop

NameError: name 'stop' is not defined

In [ ]:
import pickle
def load_model(pickles):
    '''
    Input: string name with .pickle extension. Must be present in working directory. Check with 'pwd' command
    Loads pretrained model variable from pickle file at filepath(pickle).
    '''
    with open(pickles, 'rb') as p:
        return pickle.load(p)

In [ ]:
model = load_model('basic_bert.pickle')

In [ ]:
# from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
# model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

trainer = Trainer(
    model,
    args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
x, y = dataset_test.select_columns(['input_ids','attention_mask']), dataset_test.select_columns(['labels'])

## Evaluation

#### This part is unfinished

In [ ]:
import torch
model.eval()
with torch.no_grad():
    outputs = trainer.predict(x)


In [ ]:
y

In [ ]:
len(y[0]['labels'])

In [ ]:
outputs.predictions[0][47]

In [ ]:
predictions = np.argmax(outputs.predictions, axis=2)

predictions = predictions.tolist()

In [ ]:
df_eval = pd.DataFrame()
df_eval['predictions'] = predictions
df_eval['true_labels'] = y
df_eval.to_csv('outputs/test_set_predictions.csv', index=False)

In [ ]:
import pickle
def picklify(pickles,not_so_pickles):
    '''
    Saves a model or variable to a pickle file 
    '''
    with open(pickles, 'wb') as p:
        pickle.dump(not_so_pickles, p)
picklify('basic_bert.pickle',model)

In [ ]:
df_eval

In [ ]:
df_eval

In [27]:
df_train,  df_test = pd.DataFrame(dataset_train), pd.DataFrame(dataset_test)

### Replace -100 labels with 0

In [28]:
df_test['labels'], df_train['labels'] = df_test['labels'].apply(lambda x: [e if e != -100 else 61 for e in x]), df_train['labels'].apply(lambda x: [e if e != -100 else 0 for e in x])

## Create generator for training advanced model

In [29]:
import os
def create_generator(df, folder_path, chunk_size):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    chunk_start = 0
    chunk_end = chunk_size
    chunk_count = len(df) // chunk_size + (1 if len(df) % chunk_size else 0)
    for i in range(chunk_count):
        chunk = df[chunk_start:chunk_end]
        file_name = f"chunk_{i}.csv"
        chunk.to_csv(os.path.join(folder_path, file_name), index=False)
        chunk_start += chunk_size
        chunk_end += chunk_size

create_generator(df_train, 'df_train_folder', chunk_size=50)
create_generator(df_test, 'df_test_folder', chunk_size=50)

In [30]:
df_train['input_ids'].map(lambda x: len(x)).max()

182

#### ^Due to this length we will set our padding length max_len to 200

In [31]:
max_len = 200

## Padding 

In [32]:
import os
import random
import pandas as pd
import tensorflow as tf

def data_generator(folder_path, max_len=200):
    def pad(tokens):
        tokens = tokens.replace('[', '').replace(']', '')
        tokens = tf.convert_to_tensor([int(token) for token in tokens.split(',')], dtype=tf.int32)
        if tf.size(tokens) > max_len:
            return tokens[tf.size(tokens)-max_len-1:]
        else:
            padding = tf.constant([-100 for _ in range(max_len - tf.size(tokens))], dtype=tf.int32)
            return tf.concat([tokens, padding], axis=0)
    
    def extend(attention):
        attention = [int(x) for x in attention.replace('[', '').replace(']', '').split(',')]
        return tf.convert_to_tensor(attention + [0] * (max_len - len(attention)), dtype=tf.int32)

    file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path)]
    random.shuffle(file_paths)
    for file_path in file_paths:
        batch_x = []
        batch_y = []
        with open(file_path, 'r') as f:
            df = pd.read_csv(file_path)
            # print(len(df))
            for _, row in df.iterrows():
                input_ids = pad(row['input_ids'])
                attention_mask = extend(row['attention_mask'])
                batch_x.append( tf.stack([input_ids, attention_mask], axis=1))
                batch_y.append(pad(row['labels']))
                # print( tf.stack([input_ids, attention_mask], axis=1))
        yield tf.convert_to_tensor(batch_x), tf.convert_to_tensor(batch_y)
                

In [33]:
sample = [i for i in data_generator('df_test_folder')][0]
input_dim = sample[0].shape
num_labels = sample[1].shape[1]
sample

(<tf.Tensor: shape=(50, 200, 2), dtype=int32, numpy=
 array([[[ 101,    1],
         [1045,    1],
         [2079,    1],
         ...,
         [-100,    0],
         [-100,    0],
         [-100,    0]],
 
        [[ 101,    1],
         [1045,    1],
         [2079,    1],
         ...,
         [-100,    0],
         [-100,    0],
         [-100,    0]],
 
        [[ 101,    1],
         [1045,    1],
         [2079,    1],
         ...,
         [-100,    0],
         [-100,    0],
         [-100,    0]],
 
        ...,
 
        [[ 101,    1],
         [2245,    1],
         [2008,    1],
         ...,
         [-100,    0],
         [-100,    0],
         [-100,    0]],
 
        [[ 101,    1],
         [2092,    1],
         [1010,    1],
         ...,
         [-100,    0],
         [-100,    0],
         [-100,    0]],
 
        [[ 101,    1],
         [2031,    1],
         [2017,    1],
         ...,
         [-100,    0],
         [-100,    0],
         [-100,    0]]])>,
 

In [34]:
result = [tensor[1] for tensor in data_generator('df_train_folder') if tf.reduce_any(tensor[1] > 61)]


In [35]:
result

[]

In [36]:
sample[1].shape


TensorShape([50, 200])

In [37]:
num_labels = len(set(label for sequence in df_train['labels'] for label in sequence))
num_labels

60

## Advanced model

In [38]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Bidirectional
from tf2crf import CRF, ModelWithCRFLoss
from tensorflow.keras.optimizers import Adam

def create_lstm_crf(seq_len, feature_dim, num_labels):
    print(f'seq_len:{seq_len}, feature_dim:{feature_dim}, num_labels:{num_labels}')
    input = Input(shape=(seq_len, feature_dim), dtype='float32')
    
    bilstm_layer1 = Bidirectional(LSTM(units=feature_dim, return_sequences=True))
    bilstm_output1 = bilstm_layer1(input)
    
    bilstm_layer2 = Bidirectional(LSTM(units=num_labels, return_sequences=True))
    bilstm_output2 = bilstm_layer2(bilstm_output1)


    
    crf = CRF(dtype='float32')
    output = crf(bilstm_output2)
    
    base_model = Model(input, output)
    model = ModelWithCRFLoss(base_model, sparse_target=True)

    return base_model

seq_len = sample[1].shape[1]
feature_dim = sample[0].shape[2]
num_labels = num_labels * seq_len #label for every element in the sequence

model = create_lstm_crf(seq_len, feature_dim, num_labels)
model.build(input_dim)
model.summary()

C:\Users\katko\miniconda3\Lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\katko\miniconda3\Lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.15.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure 


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200, 2)]          0         
                                                                 
 bidirectional (Bidirection  (None, 200, 4)            80        
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 200, 24000)        1152480000
 onal)                                                           
                                                                 
 crf (CRF)                   ((None, 200),             576000000 
                              (None, 200, 24000),                
                              (None,),                           
                              (24000, 24000))                    
                                                            

C:\Users\katko\miniconda3\Lib\site-packages\keras\src\utils\layer_utils.py:146: RuntimeWarning: overflow encountered in scalar multiply
  total_memory_size += weight_shape * per_param_size
C:\Users\katko\miniconda3\Lib\site-packages\keras\src\utils\layer_utils.py:146: RuntimeWarning: overflow encountered in scalar add
  total_memory_size += weight_shape * per_param_size


In [39]:
model.compile(optimizer=Adam(0.001), loss='mse')

In [40]:
history = model.fit(data_generator('df_test_folder'), batch_size=50, epochs=1, steps_per_epoch=96)

InvalidArgumentError: Graph execution error:

Detected at node mean_squared_error_2/remove_squeezable_dimensions/Squeeze defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\katko\miniconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\katko\miniconda3\Lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance

  File "C:\Users\katko\miniconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "C:\Users\katko\miniconda3\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "C:\Users\katko\miniconda3\Lib\asyncio\base_events.py", line 607, in run_forever

  File "C:\Users\katko\miniconda3\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "C:\Users\katko\miniconda3\Lib\asyncio\events.py", line 80, in _run

  File "C:\Users\katko\miniconda3\Lib\site-packages\ipykernel\kernelb

In [ ]:
set(label for sequence in df_test['labels'] for label in sequence)

In [ ]:
x_tensors = [i[0] for i in data_generator('df_test_folder')]
y_tensors = [i[1] for i in data_generator('df_test_folder')]

if len(x_tensors[-1]) != 50:
    x_tensors.pop() 
    y_tensors.pop()

x_tensors_filtered = [tensor for tensor in x_tensors if tensor.shape[0] == 50]
y_tensors_filtered = [tensor for tensor in y_tensors if tensor.shape[0] == 50]

x = tf.convert_to_tensor(x_tensors_filtered)
y = tf.convert_to_tensor(y_tensors_filtered)
x = tf.reshape(x, [-1, 200, 2])
y = tf.reshape(y, [-1, 200])
history = model.fit(x,y, batch_size=50, epochs=1)

In [ ]:
print(tf.reshape(y, [-1, 200]).shape)